#### user_coordinate_area_20~24의 end 좌표를 basic-Unit 단위로 클러스링
1. basic_unit unique 값 추출
2. 추출한 unique 값 기준으로 20~24년 end 좌표 추출
3. 실루엣 스코어 판단, k= [3, 4, 5, 6]
4. 가장 높은 실루엣 스코어 값을 기준으로 k-means 클러스터링 진행
5. 좌표에 따른 레이블 값 매핑
6. basic-unit과 cluster_label을 기준으로 uuid를 담은 컬렉션 생성

In [ ]:
from dotenv import load_dotenv
import os
from pymongo import MongoClient, UpdateOne
import pandas as pd
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score


1. basic_unit unique 값 추출

In [2]:
# 데이터베이스 연결
load_dotenv()
client = MongoClient(os.getenv('DB_ADR'),
          username=os.getenv('DB_USER'),
          password=os.getenv('DB_PASSWORD'),
          authSource=os.getenv('DB_AuthSource'),
          authMechanism=os.getenv('DB_AuthMechanism'))

db = client.get_database(os.getenv('DB_Collection'))
collection_list=['user_coordinate_area_2024','user_coordinate_area_2023',
                 'user_coordinate_area_2022','user_coordinate_area_2021',
                 'user_coordinate_area_2020']


In [3]:
basic_unit_set=set()
for col in collection_list:
    collection = db[col]
    basic_unit_set.update(collection.distinct('destination_area'))
basic_unit_set

{nan,
 nan,
 nan,
 nan,
 nan,
 '가평군',
 '강릉시',
 '강진군',
 '거제시',
 '거창군',
 '경산시',
 '경주시',
 '계룡시',
 '고령군',
 '고성군',
 '고양시 덕양구',
 '고양시 일산동구',
 '고양시 일산서구',
 '고창군',
 '고흥군',
 '곡성군',
 '공주시',
 '과천시',
 '광명시',
 '광양시',
 '광주 광산구',
 '광주 남구',
 '광주 동구',
 '광주 북구',
 '광주 서구',
 '광주시',
 '괴산군',
 '구례군',
 '구리시',
 '구미시',
 '군산시',
 '군포시',
 '금산군',
 '김제시',
 '김천시',
 '김포시',
 '김해시',
 '나주시',
 '남양주시',
 '남원시',
 '남해군',
 '논산시',
 '단양군',
 '담양군',
 '당진시',
 '대구 군위군',
 '대구 남구',
 '대구 달서구',
 '대구 달성군',
 '대구 동구',
 '대구 북구',
 '대구 서구',
 '대구 수성구',
 '대구 중구',
 '대전 대덕구',
 '대전 동구',
 '대전 서구',
 '대전 유성구',
 '대전 중구',
 '동두천시',
 '동해시',
 '목포시',
 '무안군',
 '무주군',
 '문경시',
 '밀양시',
 '보령시',
 '보성군',
 '보은군',
 '봉화군',
 '부산 강서구',
 '부산 금정구',
 '부산 기장군',
 '부산 남구',
 '부산 동구',
 '부산 동래구',
 '부산 부산진구',
 '부산 북구',
 '부산 사상구',
 '부산 사하구',
 '부산 서구',
 '부산 수영구',
 '부산 연제구',
 '부산 영도구',
 '부산 중구',
 '부산 해운대구',
 '부안군',
 '부여군',
 '부천시',
 '사천시',
 '산청군',
 '삼척시',
 '상주시',
 '서귀포시',
 '서산시',
 '서울 강남구',
 '서울 강동구',
 '서울 강북구',
 '서울 강서구',
 '서울 관악구',
 '서울 광진구',
 '서울 구로구',
 '서울 금천구',
 '서울 노원구',
 '서울

In [4]:
basic_unit_list=list(basic_unit_set)
print(basic_unit_list)
len(basic_unit_list)

['부산 영도구', '의령군', '창원시 의창구', '충주시', '나주시', '서울 종로구', '속초시', '거창군', '서울 영등포구', '안양시 동안구', '부산 중구', '서울 강북구', '대구 달서구', '영동군', '울산 울주군', '서울 중구', '강릉시', '화성시', '거제시', '용인시 수지구', '성남시 수정구', '안동시', '과천시', '여수시', '음성군', '철원군', '부산 북구', '의정부시', '임실군', '장흥군', '순천시', '금산군', '김천시', '대구 남구', '경주시', '보령시', '서울 양천구', '가평군', '무안군', '창녕군', '논산시', '청주시 상당구', '부안군', '대전 서구', '서울 중랑구', '영주시', '대구 동구', '김해시', '청주시 서원구', '단양군', '영덕군', '부산 부산진구', '인천 계양구', '부산 연제구', '담양군', '괴산군', '동해시', '서귀포시', '창원시 마산회원구', '서울 광진구', '양구군', '양주시', '대전 대덕구', '부산 서구', '포항시 남구', '부산 사하구', '동두천시', '부산 강서구', '부산 동래구', '상주시', '서천군', '태안군', nan, '영양군', '정읍시', '칠곡군', '수원시 영통구', '계룡시', '진도군', '광주 북구', '서울 강동구', '수원시 팔달구', '부산 수영구', '청주시 흥덕구', '울릉군', '광주시', '인천 미추홀구', '인천 부평구', '전주시 덕진구', '양평군', '구리시', '부산 기장군', '용인시 기흥구', '산청군', '예산군', '대전 중구', '합천군', '청송군', '완주군', '함안군', '서울 서초구', '옥천군', '문경시', '인천 강화군', '함평군', '대구 군위군', '서울 은평구', '평창군', '평택시', '의왕시', '부천시', '울산 남구', '파주시', '목포시', '용인시 처인구', '의성군', '광주 서구', '인천 동구', '제천시', '서울 마포

254

In [10]:
basic_unit_list2=['인천 연수구', '정선군', '진천군', '부산 중구', '태백시', '삼척시', '하남시', '함평군', '공주시', '천안시 동남구', '영덕군', '하동군', '부산 북구', '의왕시', '무안군', '인천 동구', '음성군', '부천시', '포천시', '임실군', '영천시', '목포시', '부산 해운대구', '창녕군', '곡성군', '상주시', '서울 성동구', '광주시', '순천시', '광주 서구', '창원시 성산구', '장성군', '익산시', '서울 강남구', '서울 성북구', '김천시', '군포시', '횡성군', '서울 용산구', '부산 영도구', '완주군', '김제시', '대구 중구', '인천 중구', '부산 동래구', '진주시', '영동군', '시흥시', '무주군', '파주시', '안산시 단원구', '함양군', '제천시', '부산 동구', '보성군', '진도군', '고양시 일산서구', '서울 금천구', '장수군', '전주시 완산구', '천안시 서북구', '강릉시', '영월군', '인제군', '경주시', '제주시', '인천 남동구', '성남시 분당구', '수원시 장안구', '부산 부산진구', '서울 구로구', '화순군', '고양시 덕양구', '대구 남구', '보령시', '홍천군', '양구군', '구미시', '의정부시', '서울 노원구', '서울 중구', '광주 광산구', '당진시', '여주시', '여수시', '정읍시', '수원시 권선구', '부산 강서구', '거제시', '울산 울주군', '서울 강서구', '서울 양천구', '청주시 청원구', '계룡시', '단양군', '서산시', '부안군', '전주시 덕진구', '청양군', '영광군']
len(basic_unit_list2)

100

2. 추출한 unique 값 기준으로 20~24년 end 좌표 추출

3. 실루엣 스코어 판단, k= [3, 4, 5, 6]

4. 가장 높은 실루엣 스코어 값을 기준으로 k-means 클러스터링 진행

5. 좌표에 따른 레이블 값 매핑

In [11]:
cluster_collection=db['cluster_info']

for area in basic_unit_list2:
    docs = []
    for col in collection_list:
        collection=db[col]
        for doc in collection.find({'destination_area':area},{'destination_lat':1, 'destination_lng':1, '_id':1}):
            docs.append({
                'collection': col,
                '_id': doc['_id'],
                'destination_lat': doc['destination_lat'],
                'destination_lng': doc['destination_lng']
            })

    df = pd.DataFrame(docs)
    print(area)
    points = df[['destination_lat', 'destination_lng']].to_numpy()
    sample_points = points[np.random.choice(points.shape[0], size=int(0.1 * points.shape[0]), replace=False)]
    cluster_list = [3, 4, 5, 6]
    best_score=-1
    best_cluster=None

    for c in cluster_list:
        # 클러스터링
        MBKMeans = MiniBatchKMeans(n_clusters=c, batch_size=4096, random_state=42)
        MBKMeans.fit(sample_points)

        # 클러스터 포인트 예측
        cluster_point = MBKMeans.predict(sample_points)

        # 실루엣 스코어 계산
        score = silhouette_score(sample_points, cluster_point)

        if score > best_score:
            best_score = score
            best_cluster = c

    print(f"{area}: {best_cluster}")

    MBKMeans = MiniBatchKMeans(n_clusters=best_cluster, batch_size=2048, random_state=42)
    cluster_point=MBKMeans.fit_predict(df[['destination_lat', 'destination_lng']])
    df['cluster_label']=cluster_point

    for collection_name, group_df in df.groupby('collection'):
        bulk_ops = []
        for _, row in group_df.iterrows():
            bulk_ops.append(
                UpdateOne(
                    {'_id': row['_id']},
                    {'$set': {'cluster_label': row['cluster_label']}}
                )
            )

        if bulk_ops:
            db[collection_name].bulk_write(bulk_ops)

    
    centers = MBKMeans.cluster_centers_.tolist()
    center_points = {str(i): [float(centers[i][0]), float(centers[i][1])] for i in range(len(centers))}

    update_data = {
    "basic-unit": area,
    "cluster_n": best_cluster,
    "center_points": center_points
    }
    cluster_collection.insert_one(update_data)

    print(f"{area}: {len(df)}개 문서 업데이트")

    

인천 연수구
인천 연수구: 3
인천 연수구: 864992개 문서 업데이트
정선군
정선군: 6
정선군: 117300개 문서 업데이트
진천군
진천군: 6
진천군: 489102개 문서 업데이트
부산 중구
부산 중구: 3
부산 중구: 106010개 문서 업데이트
태백시
태백시: 6
태백시: 95375개 문서 업데이트
삼척시
삼척시: 3
삼척시: 196576개 문서 업데이트
하남시
하남시: 3
하남시: 714321개 문서 업데이트
함평군
함평군: 3
함평군: 106985개 문서 업데이트
공주시
공주시: 5
공주시: 955250개 문서 업데이트
천안시 동남구
천안시 동남구: 3
천안시 동남구: 981277개 문서 업데이트
영덕군
영덕군: 4
영덕군: 107440개 문서 업데이트
하동군
하동군: 5
하동군: 106377개 문서 업데이트
부산 북구
부산 북구: 4
부산 북구: 364366개 문서 업데이트
의왕시
의왕시: 3
의왕시: 393626개 문서 업데이트
무안군
무안군: 5
무안군: 188868개 문서 업데이트
인천 동구
인천 동구: 6
인천 동구: 171012개 문서 업데이트
음성군
음성군: 5
음성군: 611993개 문서 업데이트
부천시
부천시: 5
부천시: 1307597개 문서 업데이트
포천시
포천시: 5
포천시: 601568개 문서 업데이트
임실군
임실군: 5
임실군: 115337개 문서 업데이트
영천시
영천시: 3
영천시: 402796개 문서 업데이트
목포시
목포시: 6
목포시: 481170개 문서 업데이트
부산 해운대구
부산 해운대구: 5
부산 해운대구: 570467개 문서 업데이트
창녕군
창녕군: 6
창녕군: 234778개 문서 업데이트
곡성군
곡성군: 5
곡성군: 78500개 문서 업데이트
상주시
상주시: 3
상주시: 372095개 문서 업데이트
서울 성동구
서울 성동구: 3
서울 성동구: 535166개 문서 업데이트
광주시
광주시: 5
광주시: 1556333개 문서 업데이트
순천시
순천시: 6
순천시: 678955개 문서 업데이트
광주 서구
광주 서구:

6. basic-unit과 cluster_label을 기준으로 uuid 카운팅